# 📚 Import Libraries

In [ ]:
import pandas as pd, numpy as np, matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from geopy.distance import great_circle
from shapely.geometry import MultiPoint

%matplotlib inline
import pandas as pd
import numpy as np
from IPython.core.interactiveshell import InteractiveShell
import random
InteractiveShell.ast_node_interactivity = "all"
import os
from datetime import datetime
import matplotlib.pyplot as plt
import plotly.express as px
import glob
from tqdm import tqdm

import plotly.figure_factory as ff
import plotly.express as px
import numpy as np

import geopandas as gpd
from shapely.geometry import Point
from shapely.ops import nearest_points

### Use mapbox for visualization

In [ ]:
# use your own mapbox token here to visualize
MAPBOX_TOKEN = 'pk.eyJ1IjoiaWdsYXdlYiIsImEiOiJja3picmk5NmsyaDZxMndtenYyOWhvNmtnIn0.Dxi29pChSrUbePq_oZ1rTw'
px.set_mapbox_access_token(MAPBOX_TOKEN)

In [ ]:
# data description https://www.ncei.noaa.gov/data/global-summary-of-the-day/doc/readme.txt
aus_fire = pd.read_csv('./wildfiredataset/australia_fire_total_ready.csv')
aus_fire.shape
aus_fire.head()

In [ ]:
aus_fire.latitude.min(), aus_fire.latitude.max()
aus_fire.longitude.min(), aus_fire.longitude.max()

In [ ]:
# data description https://www.ncei.noaa.gov/data/global-summary-of-the-day/doc/readme.txt
aus_weather = pd.read_csv('./wildfiredataset/australia_weather_full2.csv', parse_dates=['DATE'])
aus_weather.shape
aus_weather.head()

# Set precision for geo coordinates

In [ ]:
# http://wiki.gis.com/wiki/index.php/Decimal_degrees
PRECISION = 2 # 2 places - 1 km
aus_weather.LATITUDE = aus_weather.LATITUDE.astype(float).round(PRECISION)
aus_weather.LONGITUDE = aus_weather.LONGITUDE.astype(float).round(PRECISION)
aus_weather.head()

# Analyze duplicates

In [ ]:
# Selecting duplicate rows based
# on list of column names
aus_weather_d = aus_weather[aus_weather.duplicated(['DATE', 'LATITUDE', 'LONGITUDE'])]
# df.drop_duplicates(subset=['DATE', 'LATITUDE', 'LONGITUDE'], keep=False, inplace=True)
print(aus_weather_d.shape)
aus_weather_d.head()

In [ ]:
print('Number of unique stations', len(aus_weather['STATION'].unique()))

In [ ]:
print('Lat range', aus_weather['LATITUDE'].min(), aus_weather['LATITUDE'].max())
print('Lng range', aus_weather['LONGITUDE'].min(), aus_weather['LONGITUDE'].max())

# Remove stations that have less than 108 measurements

In [ ]:
aus_weather['year'] = aus_weather.DATE.dt.year
aus_weather['month'] = aus_weather.DATE.dt.month
#aus_weather.latitude = aus_fires.latitude.round(PRECISION)
#aus_weather.longitude = aus_fires.longitude.round(PRECISION)
#fires = aus_weather.groupby(['STATION', 'LATITUDE', 'LONGITUDE', 'year', 'month']).size().reset_index()

#aus_wth_agg = aus_weather.groupby(['STATION', 'LATITUDE', 'LONGITUDE', 'year', 'month']) \
 #   .agg({'TEMP':'mean','DEWP':'sum', 'WDSP':'sum', 'MAX':'max', 'MIN': 'min'}) \
  #  .reset_index()

aus_wth_agg = aus_weather.groupby(['STATION', 'LATITUDE', 'LONGITUDE', 'year', 'month']) \
    .agg(T_MAX=('MAX', 'max'), T_MAX_MEAN=('MAX', 'mean'), T_MEAN=('TEMP', 'mean'), \
         DEWP_MEAN=('DEWP', 'mean'), WDSP_MEAN=('WDSP', 'mean'), \
         MXSPD_MAX=('MXSPD', 'max')) \
    .reset_index()

#fires.columns = ['latitude', 'longitude', 'year', 'month', 'temp_avg']

aus_wth_agg.shape
aus_wth_agg.head()
aus_wth_agg.nunique()

# Detect stations and its number of observations

In [ ]:
st_cnt_sample = aus_wth_agg.groupby(['STATION','LATITUDE','LONGITUDE']).size().reset_index().rename(columns={0:'count'})

st_cnt_sample = st_cnt_sample.reset_index()
st_cnt_sample = st_cnt_sample.rename(columns={"index":"STAT_ID"})
st_cnt_sample['STAT_ID'] = st_cnt_sample.index + 1

st_cnt_sample.shape
print('Max observations', st_cnt_sample['count'].max())
print('Min observations', st_cnt_sample['count'].min())
#st_cnt_sample.head()
# how many stations have only one observation
st_cnt_sample[st_cnt_sample['count'] == 1].shape
st_cnt_sample.head()

In [ ]:
# remote stations that do not have 12 * 9 = 108 month data (up to december 2021)
st_cnt_sample_108 = st_cnt_sample[st_cnt_sample['count'] >= 108]
st_cnt_sample_108.shape

In [ ]:
# drop non relevant stations
st_cnt_sample_del = st_cnt_sample[st_cnt_sample['count'] < 108]
st_cnt_sample = st_cnt_sample.drop(st_cnt_sample_del.index)

st_cnt_sample.shape
st_cnt_sample_del.head()

In [ ]:
# remove stations that have less count observations
print('Remove stations', len(st_cnt_sample_del['STATION'].unique()))

# remove stations that are in st_cnt_sample_del
cond = aus_wth_agg['STATION'].isin(st_cnt_sample_del['STATION'])
aus_wth_agg.drop(aus_wth_agg[cond].index, inplace = True)

aus_wth_agg.shape
aus_wth_agg.head()

In [ ]:
st_cnt_sample.shape
aus_wth_agg[aus_wth_agg['STATION'] == 94100099999].head(2)

# Check duplicates again

In [ ]:
# Selecting duplicate rows based
# on list of column names
# sort it first to remove rows relevant to same stations
aus_wth_agg.sort_values(by=['STATION'], ascending=True, inplace=True)
aus_wth_agg_d = aus_wth_agg[aus_wth_agg.duplicated(['year', 'month', 'LATITUDE', 'LONGITUDE'], keep=False)]

print('Before delete', aus_wth_agg.shape)
aus_wth_agg.drop_duplicates(subset=['year', 'month', 'LATITUDE', 'LONGITUDE'], keep='first', inplace=True)
print('After delete', aus_wth_agg.shape)

#aus_weather_d.head()

test = aus_wth_agg_d[(aus_wth_agg_d['year'] == 2013) & (aus_wth_agg_d['month'] == 1) & \
                     (aus_wth_agg_d['LATITUDE'] == -32.22)]
test.head()

In [ ]:
# remove stations that have less count observations
print('Stations', len(st_cnt_sample['STATION'].unique()))

# remove stations that are in st_cnt_sample_del
cond = ~st_cnt_sample['STATION'].isin(aus_wth_agg['STATION'])
st_cnt_sample.drop(st_cnt_sample[cond].index, inplace = True)

st_cnt_sample.shape
st_cnt_sample.head()

In [ ]:
import plotly.express as px
import geopandas as gpd

fig = px.scatter_geo(st_cnt_sample,
                    lat=st_cnt_sample.LATITUDE,
                    lon=st_cnt_sample.LONGITUDE,
                    hover_name="STAT_ID")
fig.show()

In [ ]:
st_cnt_sample_np = st_cnt_sample[['LATITUDE', 'LONGITUDE']].to_numpy()
st_cnt_sample_np[:10]

In [ ]:
coords = st_cnt_sample_np
kms_per_radian = 6371.0088
epsilon = 50 / kms_per_radian
db = DBSCAN(eps=epsilon, min_samples=1, algorithm='ball_tree', metric='haversine').fit(np.radians(coords))
cluster_labels = db.labels_
num_clusters = len(set(cluster_labels))
clusters = pd.Series([coords[cluster_labels == n] for n in range(num_clusters)])
print('Number of clusters: {}'.format(num_clusters))

In [ ]:
kms_per_radian = 6371.0088

AUS_LAT_RANGE = (-40, -9)
AUS_LON_RANGE = (112, 154.7)

AUS_LAT_RANGE_R = (9, 40)
AUS_LON_RANGE_R = (112, 154.7)

bottomLeft = (AUS_LAT_RANGE_R[1], AUS_LON_RANGE[0])
bottomRight = (AUS_LAT_RANGE_R[1], AUS_LON_RANGE[1])
topLeft = (AUS_LAT_RANGE_R[0], AUS_LON_RANGE[0])
topRight = (AUS_LAT_RANGE_R[0], AUS_LON_RANGE[1])

In [ ]:
# remove non relevent weather data
print('Before', aus_weather.shape)

aus_weather = aus_weather[
  (aus_weather.LATITUDE <= AUS_LAT_RANGE[1]) & (aus_weather.LATITUDE >= AUS_LAT_RANGE[0])]
aus_weather = aus_weather[
    (aus_weather.LONGITUDE <= AUS_LON_RANGE[1]) & (aus_weather.LONGITUDE >= AUS_LON_RANGE[0])]

print('After', aus_weather.shape)

In [ ]:
from geopy.distance import geodesic

coords_1 = (AUS_LAT_RANGE[1], AUS_LON_RANGE[0])
coords_2 = (AUS_LAT_RANGE[0], AUS_LON_RANGE[0])
lat_dist = geodesic(coords_1, coords_2).km
print('Lat dist', lat_dist)

coords_1 = (AUS_LAT_RANGE[1], AUS_LON_RANGE[0])
coords_2 = (AUS_LAT_RANGE[1], AUS_LON_RANGE[1])
lng_dist = geodesic(coords_1, coords_2).km
print('Lng dist', lng_dist)

#coords_1 = (-9, 40)
#coords_2 = (-9, 41)
#lng_dist = geodesic(coords_1, coords_2).km
#print('Lng dist', lng_dist)

## Bins and weather stations

We cannot process entire dataset and match weather stations due to platform memory limits. The datasets of fire and weather records is relatively big. The idea that I came up with is to divide the Australia mainland into grid cells (see bins here) and assign a weather station located in this bin to every fire record that is inside this bin. This approach helps us not to iterate through all combinations of fire records and weather stations, and save memory and computations resources.

In [ ]:
cnt_rows = lat_dist / 100
cnt_cols = lng_dist / 100

rows = np.linspace(bottomLeft[1], bottomRight[1], num=int(cnt_rows))
cols = np.linspace(topLeft[0], bottomLeft[0], num=int(cnt_cols))

cols_gap = abs(cols[1] - cols[0])
rows_gap = abs(rows[1] - rows[0])
print('Cols cnt', len(cols))
print('Rows cnt', len(rows))
print('Cols gap', cols_gap, 'Rows gap', rows_gap)
print(rows)
print(cols)

print(np.linspace(1, 10, num=3))

def detect_bin(lat, lng):
    col_lng = (int)((lng - AUS_LON_RANGE_R[0]) / cols_gap)
    col_lat = (int)((abs(lat) - AUS_LAT_RANGE_R[0]) / rows_gap)
    return f'{col_lat},{col_lng}'

In [ ]:
# check stations inside bin
non_null_first = 0
dict_w = {}
for idx1, r in enumerate(tqdm(rows)):
    for idx2, c in enumerate(tqdm(cols)):
        if idx1 == 0 or idx2 == 0: continue
        #print(r, c)
        col_lat1 = cols[idx2 - 1]
        col_lat2 = cols[idx2]
        rows1_lng = rows[idx1 - 1]
        rows2_lng = rows[idx1]
        
        #if idx1 < 3 and idx2 < 3:
         #   print(-col_lat1, -col_lat2)
         #   print(rows1_lng, rows2_lng)
        
        aus_1 = st_cnt_sample[
          (st_cnt_sample.LATITUDE <= -col_lat1) & (st_cnt_sample.LATITUDE >= -col_lat2)]
        aus_1 = aus_1[
            (aus_1.LONGITUDE <= rows2_lng) & (aus_1.LONGITUDE >= rows1_lng)]
        
        if non_null_first == 0 and len(aus_1.index) > 0:
            print('First ', (idx1, idx2), (r, c), len(aus_1.index))
            non_null_first = len(aus_1.index)
        dict_w[(idx1, idx2)] = len(aus_1.index)

print(max(dict_w, key=dict_w.get))        
print(dict_w)

In [ ]:
st_cnt_sample['st_bin'] = st_cnt_sample.apply(lambda x: detect_bin(x.LATITUDE, x.LONGITUDE), axis=1)
st_cnt_sample.shape
st_cnt_sample.head()

In [ ]:
aus_wth_agg['st_bin'] = aus_wth_agg.apply(lambda x: detect_bin(x.LATITUDE, x.LONGITUDE), axis=1)
aus_wth_agg.head()

In [ ]:
el = aus_wth_agg['st_bin'].unique()
len(el)
aus_wth_agg['st_bin'].values
aus_wth_agg.shape

In [ ]:
aus_wth_agg.to_csv("aus_weather_binned_new.csv", index=False)
print('Submission saved')

In [ ]:
aus_fires = pd.read_csv('./australia_fire_total_ready.csv')
aus_fires.shape
aus_fires.head()

In [ ]:
aus_fires['st_bin'] = aus_fires.apply(lambda x: detect_bin(x.latitude, x.longitude), axis=1)
aus_fires.head()

In [ ]:
fire_st_list = aus_fires['st_bin'].unique()
len(fire_st_list)
print(fire_st_list[:10])
print(dict_w.get((31,33)))

no_st = []
ret = []
for idx1, r in enumerate (rows):
    for idx2, c in enumerate(cols):
        dict_val = dict_w.get((idx1, idx2))
        if dict_val == 0: # no station in this zone
            no_st.append((idx1, idx2))

        if f'{idx1},{idx2}' not in fire_st_list:
            ret.append(f'{idx1},{idx2}')
print('Missing zones', len(ret))

need_st = []
for st_b in tqdm(fire_st_list):
    idx1, idx2 = st_b.split(',')
    idx1 = int(idx1)
    idx2 = int(idx2)
    
    dict_val = dict_w.get((idx1, idx2))
    if dict_val == 0: # no station in this zone
        need_st.append((idx1, idx2))


print('No weather stations', len(need_st))

### We are ready to save binned weather dataset

In [ ]:
aus_fires.to_csv("aus_fires_binned.csv", index=False)
print('Submission saved')

### Find nearest weather stations for every fire record

Use GeoPandas package for this task https://geopandas.org/en/stable/docs/reference/api/geopandas.sindex.SpatialIndex.nearest.html

In [ ]:
import geopandas as gpd
from shapely.geometry import Point
from shapely.ops import nearest_points

geometry = [Point(xy) for xy in zip(aus_wth_agg.LONGITUDE, aus_wth_agg.LATITUDE)]
gdf = gpd.GeoDataFrame(aus_wth_agg, crs="EPSG:4326", geometry=geometry)

aus_fires['near_st_lat'] = None
aus_fires['near_st_lng'] = None
multipoint = gdf.drop(index, axis=0).geometry.unary_union

for index, row in tqdm(aus_fires.iterrows()):
    point = Point(row.longitude, row.latitude)
    queried_geom, nearest_geom = nearest_points(point, multipoint)
    aus_fires.at[index, 'near_st_lng'] = nearest_geom.x #nearest_geom
    aus_fires.at[index, 'near_st_lat'] = nearest_geom.y #nearest_geom

aus_fires.head()

### Save a dataset of fire records with information about weather stations

In [ ]:
aus_fires.to_csv("aus_fires_binned_geometry.csv", index=False)
print('Submission saved')